In [1]:
import requests
import json
import pandas as pd
from pandas import json_normalize
from google.protobuf.json_format import Parse, MessageToJson
from google.protobuf.timestamp_pb2 import Timestamp
from phenopackets import Phenopacket, Individual, PhenotypicFeature, OntologyClass, GeneDescriptor, GenomicInterpretation, Diagnosis, Interpretation

In [2]:
data = pd.read_csv('case.csv')
data.head()

,id,patient_id,gene_id,features
0,patient_1,Patient:SCV000492960,Entrez:10000,"HP:0005306,HP:0000256,HP:0001263,HP:0002126"
1,patient_2,Patient:SCV000747622,Entrez:10002,"HP:0000505,HP:0000666,HP:0000551,HP:0000556"
2,patient_3,Patient:SCV000599197,Entrez:10002,HP:0000556
3,patient_4,Patient:SCV000598737,Entrez:1001,HP:0007754
4,patient_5,Patient:293197,Entrez:100151683,"HP:0001631,HP:0001601,HP:0011344,HP:0011994"


In [3]:
dj = data.to_json(orient = 'records')
parsed = json.loads(dj)

In [4]:
parsed[0]

{'id': 'patient_1',
 'patient_id': 'Patient:SCV000492960',
 'gene_id': 'Entrez:10000',
 'features': 'HP:0005306,HP:0000256,HP:0001263,HP:0002126'}

In [5]:
#test
from google.protobuf.json_format import Parse, MessageToJson
from google.protobuf.timestamp_pb2 import Timestamp
from phenopackets import Phenopacket, Individual, PhenotypicFeature, OntologyClass

# Parsing phenopackets from json
# with open('file.json', 'r') as jsfile:
#     phenopacket = Parse(Phenopacket(), text=jsfile.read())

# Writing phenopackets to json
with open('file.json', 'w') as jsfile:
    subject = Individual(id="Zaphod", sex="MALE", date_of_birth=Timestamp(seconds=-123456798))
    phenotypic_features = [PhenotypicFeature(type=OntologyClass(id="HG2G:00001", label="Hoopy")),
                           PhenotypicFeature(type=OntologyClass(id="HG2G:00002", label="Frood"))]

    phenopacket = Phenopacket(id="PPKT:1", subject=subject, phenotypic_features=phenotypic_features)

    json = MessageToJson(phenopacket)
    jsfile.write(json)

In [6]:
with open('example.json', 'w') as jsfile:
    subject = Individual(id=parsed[0]['patient_id'])
    phenotypic_features = [PhenotypicFeature(type=OntologyClass(id=parsed[0]['features'].split(',')[0])),
                           PhenotypicFeature(type=OntologyClass(id=parsed[0]['features'].split(',')[1]))]
    gene_descriptor = GeneDescriptor(symbol = parsed[0]['gene_id'])
    genomic_interpretations = GenomicInterpretation(gene=gene_descriptor)
    diagnosis = Diagnosis(genomic_interpretations = [genomic_interpretations])
    interpretations = [Interpretation(diagnosis=diagnosis)]
    
    phenopacket = Phenopacket(id=parsed[0]['id'], subject=subject, phenotypic_features=phenotypic_features, interpretations=interpretations)

    json = MessageToJson(phenopacket)
    jsfile.write(json)

In [7]:
for i in range(0, len(parsed)):
    y = parsed[i]['id']
    x = parsed[i]['patient_id']
    z = parsed[i]['features'].split(',')[0]
    w = parsed[i]['gene_id']
    value = z.split(',')
    features = []
    for j in range(0, len(value)):
        features.append(PhenotypicFeature(type=OntologyClass(id=value[j])))
        
    num = str(i+1)
    with open('patient'+num+'.json', 'w') as jsfile:
        subject = Individual(id = x)
        phenotypic_features = features
        
        gene_descriptor = GeneDescriptor(symbol = w)
        genomic_interpretations = GenomicInterpretation(gene=gene_descriptor)
        diagnosis = Diagnosis(genomic_interpretations = [genomic_interpretations])
        interpretations = [Interpretation(diagnosis=diagnosis)]


        phenopacket = Phenopacket(id = y, subject=subject, phenotypic_features=phenotypic_features, interpretations=interpretations)

        json = MessageToJson(phenopacket)
        jsfile.write(json)